In [1]:
import ast

def get_variable_dependencies(source_code):
    tree = ast.parse(source_code)
    dependencies = {}

    global_vars = set()  # To track global variables

    class DependencyAnalyzer(ast.NodeVisitor):
        def visit_Assign(self, node):
            # Get the variable names on the left-hand side
            if isinstance(node.targets[0], ast.Name):
                var_name = node.targets[0].id
                # Get variable dependencies from the right-hand side
                used_vars = set()
                for subnode in ast.walk(node.value):
                    if isinstance(subnode, ast.Name) and isinstance(subnode.ctx, ast.Load):
                        used_vars.add(subnode.id)

                dependencies[var_name] = {"depends_on": used_vars}
            
            self.generic_visit(node)

        def visit_FunctionDef(self, node):
            # Skip function bodies since they are not in __main__
            pass

    # Collect global variables
    class GlobalVarCollector(ast.NodeVisitor):
        def visit_Assign(self, node):
            if isinstance(node.targets[0], ast.Name):
                global_vars.add(node.targets[0].id)
            self.generic_visit(node)

    # First, collect all global variables
    GlobalVarCollector().visit(tree)

    # Then, analyze dependencies
    DependencyAnalyzer().visit(tree)

    # Mark whether dependencies were defined in __main__ or not
    for var, info in dependencies.items():
        info["defined_in_main"] = var in global_vars
        info["depends_on_in_main"] = {var for var in info["depends_on"] if var in global_vars}
        info["depends_on_outside_main"] = info["depends_on"] - info["depends_on_in_main"]

    return dependencies

# Example usage:
code = """
x = 5
y = x + 2
z = y * a
def func():
    b = 3
    return b + x
"""

dependencies = get_variable_dependencies(code)
for var, info in dependencies.items():
    print(f"Variable '{var}':")
    print(f"  Depends on: {info['depends_on']}")
    print(f"  Defined in __main__: {info['defined_in_main']}")
    print(f"  Dependencies in main: {info['depends_on_in_main']}")
    print(f"  Dependencies outside main: {info['depends_on_outside_main']}")
    print()

Variable 'x':
  Depends on: set()
  Defined in __main__: True
  Dependencies in main: set()
  Dependencies outside main: set()

Variable 'y':
  Depends on: {'x'}
  Defined in __main__: True
  Dependencies in main: {'x'}
  Dependencies outside main: set()

Variable 'z':
  Depends on: {'y', 'a'}
  Defined in __main__: True
  Dependencies in main: {'y'}
  Dependencies outside main: {'a'}



In [2]:
import inspect

def get_variable_dependencies_runtime(var_name, global_scope):
    if var_name not in global_scope:
        raise ValueError(f"Variable '{var_name}' is not defined.")

    code_lines = inspect.getsourcelines(global_scope[var_name])[0] if inspect.isfunction(global_scope[var_name]) else []
    
    dependencies = set()
    for line in code_lines:
        tokens = line.split()
        for token in tokens:
            if token in global_scope and token != var_name:
                dependencies.add(token)

    return dependencies

# Example usage
x = 5
y = x + 2
z = y * 3

# Check dependencies
print("Dependencies of 'y':", get_variable_dependencies_runtime('y', globals()))
print("Dependencies of 'z':", get_variable_dependencies_runtime('z', globals()))

Dependencies of 'y': set()
Dependencies of 'z': set()


In [3]:
import inspect

# Global variables
GLOBAL_VAR_1 = 10
GLOBAL_VAR_2 = "Hello"

def some_function():
    return GLOBAL_VAR_1 + 5  # This function depends on GLOBAL_VAR_1

def another_function():
    print(GLOBAL_VAR_2)  # This function depends on GLOBAL_VAR_2

def analyze_function_dependencies(func):
    """Analyze function dependencies on global variables."""
    closure_vars = inspect.getclosurevars(func)
    return closure_vars.globals  # Returns the global variables referenced in the function

# Check dependencies
print("Dependencies of some_function:", analyze_function_dependencies(some_function))
print("Dependencies of another_function:", analyze_function_dependencies(another_function))

Dependencies of some_function: {'GLOBAL_VAR_1': 10}
Dependencies of another_function: {'GLOBAL_VAR_2': 'Hello'}


In [4]:
def analyze_function_globals(func):
    """Check if a function depends on variables defined in __main__."""
    closure_vars = inspect.getclosurevars(func).globals
    dependencies = set(closure_vars.keys())

    results = {}
    for var in dependencies:
        var_obj = globals().get(var, None)
        if var_obj is not None:
            # Try to get __module__ if available
            var_module = getattr(var_obj, "__module__", None)
            if var_module is None:  
                # If no __module__, assume it's defined in __main__ if it's a built-in type
                var_module = "__main__" if type(var_obj).__module__ == "builtins" else "imported"

            results[var] = var_module == "__main__"  # True if from main, False if imported

    return {
        "function": func.__name__,
        "dependencies": dependencies,
        "defined_in_main": {var for var, is_main in results.items() if is_main},
        "imported": {var for var, is_main in results.items() if not is_main}
    }
# Analyze functions
print(analyze_function_globals(some_function))

{'function': 'some_function', 'dependencies': {'GLOBAL_VAR_1'}, 'defined_in_main': {'GLOBAL_VAR_1'}, 'imported': set()}


In [5]:
from module3 import a
import numpy as np


def f():
    np.sum
    print(a)
print(analyze_function_globals(f))

{'function': 'f', 'dependencies': {'a', 'np'}, 'defined_in_main': {'a', 'np'}, 'imported': set()}


In [6]:
type(a)

int

In [7]:
import __main__
b = 5

__main__.__dict__['a']

5